In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

import nest_asyncio


sys.path.insert(0, os.path.abspath('..'))
nest_asyncio.apply()

In [2]:
import logging


logging.basicConfig(
    level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s'
)

In [3]:
from math_rag.infrastructure.containers import InfrastructureContainer


infrastructure_container = InfrastructureContainer()
infrastructure_container.init_resources()

math_article_seeder = infrastructure_container.math_article_seeder()
math_expression_seeder = infrastructure_container.math_expression_seeder()
math_expression_prediction_seeder = (
    infrastructure_container.math_expression_prediction_seeder()
)
math_article_seeder.seed()
await math_expression_seeder.seed()
await math_expression_prediction_seeder.seed()

math_article_repository = infrastructure_container.math_article_repository()
math_expression_repository = infrastructure_container.math_expression_repository()
math_expression_prediction_repository = (
    infrastructure_container.math_expression_prediction_repository()
)
google_file_repository = infrastructure_container.google_file_repository()

2025-03-02 21:55:21,785 - INFO - file_cache is only supported with oauth2client<4.0.0


In [4]:
import gzip
import shutil
import tarfile

from io import BytesIO
from pathlib import Path
from zipfile import ZipFile

from IPython.display import Math, display
from pylatexenc.latexwalker import LatexMathNode


ARTICLES_PATH = '../tmp/articles'

### Extract

In [ ]:
def get_gzip_original_filename(file_path):
    with open(file_path, 'rb') as f:
        if f.read(2) != b'\x1f\x8b':
            return None
        f.read(1)
        flag = f.read(1)[0]
        f.read(4)
        f.read(1)
        f.read(1)
        orig_name = None
        if flag & 0x08:
            name_bytes = bytearray()
            while True:
                b = f.read(1)
                if not b or b == b'\x00':
                    break
                name_bytes.extend(b)
            try:
                orig_name = name_bytes.decode('utf-8')
            except UnicodeDecodeError:
                orig_name = name_bytes.decode('latin1')
        return orig_name


def extract_gz(file_path, dest_folder):
    orig_name = get_gzip_original_filename(file_path)
    if not orig_name:
        orig_name = os.path.splitext(os.path.basename(file_path))[0]
    dest_path = os.path.join(dest_folder, orig_name)
    with gzip.open(file_path, 'rb') as f_in, open(dest_path, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)


def extract_tar_gz(file_path, dest_folder):
    with tarfile.open(file_path, 'r:gz') as tar:
        tar.extractall(path=dest_folder)


def process_subdir(subdir_path):
    files = os.listdir(subdir_path)
    pdf_files = {f for f in files if f.endswith('.pdf')}
    for pdf in pdf_files:
        base_name = pdf[:-4]
        gz_name = f'arXiv-{base_name}.gz'
        tar_gz_name = f'arXiv-{base_name}.tar.gz'
        gz_file = None
        if tar_gz_name in files:
            gz_file = tar_gz_name
        elif gz_name in files:
            gz_file = gz_name
        if gz_file:
            new_dir = os.path.join(subdir_path, base_name)
            os.makedirs(new_dir, exist_ok=True)
            shutil.move(os.path.join(subdir_path, pdf), new_dir)
            shutil.move(os.path.join(subdir_path, gz_file), new_dir)
            new_gz_path = os.path.join(new_dir, gz_file)
            if gz_file.endswith('.tar.gz'):
                extract_tar_gz(new_gz_path, new_dir)
            else:
                extract_gz(new_gz_path, new_dir)


def extract_all():
    for subdir in os.listdir(ARTICLES_PATH):
        subdir_path = os.path.join(ARTICLES_PATH, subdir)
        if os.path.isdir(subdir_path):
            process_subdir(subdir_path)


extract_all()

/tmp/ipykernel_15043/154124979.py:39: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(path=dest_folder)


In [9]:
def clean():
    for root, dirs, files in os.walk(ARTICLES_PATH):
        for file in files:
            if file.endswith('.gz'):
                os.remove(os.path.join(root, file))


clean()

### Load

In [ ]:
folder_name = 'articles'
name = 'articles_v1.zip'  # NOTE

file_id = google_file_repository.get_file_id(name, folder_name)
assert file_id is not None

file_bytes = google_file_repository.get_file_by_id(file_id)

In [15]:
with ZipFile(file_bytes, 'r') as zip_file:
    files = {
        name: zip_file.read(name)
        for name in zip_file.namelist()
        if not name.endswith('/')
    }

In [16]:
from math_rag.core.models import MathArticle


math_articles = [MathArticle(name=name, bytes=bytes) for name, bytes in files.items()]
math_article_repository.insert_math_articles(math_articles)

In [ ]:
for name in math_article_repository.list_math_article_names():
    print(name)

articles_v1/ac/2502.13273v1/2502.13273v1.pdf
articles_v1/ac/2502.13273v1/AlgClos14.tex
articles_v1/ac/2502.13276v1/2502.13276v1.pdf
articles_v1/ac/2502.13276v1/Main.tex
articles_v1/ac/2502.13806v1/2502.13806v1.pdf
articles_v1/ac/2502.13806v1/arXiv.bbl
articles_v1/ac/2502.13806v1/arXiv.tex
articles_v1/ac/2502.13903v1/2502.13903v1.pdf
articles_v1/ac/2502.13903v1/dpsl2_20250219.tex
articles_v1/ag/2502.13800v1/2502.13800v1.pdf
articles_v1/ag/2502.13800v1/neron.tex
articles_v1/ag/2502.13815v1/2502.13815v1.pdf
articles_v1/ag/2502.13815v1/Z3-v3.tex
articles_v1/ag/2502.13882v1/2502.13882v1.pdf
articles_v1/ag/2502.13882v1/main.tex
articles_v1/ag/2502.13892v1/2502.13892v1.pdf
articles_v1/ag/2502.13892v1/_commands.tex
articles_v1/ag/2502.13892v1/_preamble.tex
articles_v1/ag/2502.13892v1/abstract.tex
articles_v1/ag/2502.13892v1/appl.tex
articles_v1/ag/2502.13892v1/cl.tex
articles_v1/ag/2502.13892v1/faces.tex
articles_v1/ag/2502.13892v1/filt.tex
articles_v1/ag/2502.13892v1/intro.tex
articles_v1/ag/

### Parse

In [10]:
from math_rag.infrastructure.services import LatexService


latex_service = LatexService()

In [ ]:
file_names = math_article_repository.list_math_article_names()
file_names = [x for x in file_names if x.endswith('.tex')]

math_nodes_by_file_name: dict[str, list[LatexMathNode]] = {}

for name in file_names:
    math_article = math_article_repository.get_math_article_by_name(name)
    latex = latex_service.read(math_article.bytes)
    nodes = latex_service.parse(latex)

    math_nodes_by_file_name[name] = []
    append_math_node = lambda x: math_nodes_by_file_name[name].append(x)
    callbacks = {LatexMathNode: append_math_node}

    latex_service.traverse(nodes, callbacks)

In [15]:
import re

from math_rag.core.enums import MathCategory
from math_rag.core.models import MathExpression


math_expressions: list[MathExpression] = []

for file_name, math_nodes in math_nodes_by_file_name.items():
    category_slug = re.search(r'articles_v\d+/([^/]+)/', file_name).group(1)

    for math_node in math_nodes:
        category = MathCategory.from_str(category_slug)
        math_expression = MathExpression(
            latex=math_node.latex_verbatim(),
            position=math_node.pos,
            is_inline=math_node.displaytype == 'inline',
            math_category=category,
        )
        math_expressions.append(math_expression)

In [16]:
await math_expression_repository.insert_math_expressions(math_expressions)

### Display

In [10]:
def fix_latex(latex_str: str):
    fixed = latex_str.replace('\\[', '$$').replace('\\]', '$$')
    fixed = fixed.replace('\\EE', '\\mathbb{E}')
    fixed = fixed.replace('\\II', '\\mathbb{I}')
    fixed = fixed.replace('\\Var', '\\mathrm{Var}')
    fixed = fixed.replace('\\HH', '\\mathbb{H}')
    fixed = fixed.replace('\\AND', '\\wedge')
    fixed = fixed.replace('\\OR', '\\vee')
    fixed = fixed.replace('\\mathbbm{1}', '\\mathbf{1}')
    fixed = fixed.replace('\\Maj', '\\mathrm{Maj}')
    fixed = fixed.replace('\\sgn', '\\operatorname{sgn}')
    fixed = fixed.replace('\\Tribus', '\\mathrm{Tribus}')
    fixed = fixed.replace('\\linebreak', '\\text{ }')
    fixed = fixed.replace('\\Prob', '\\mathbb{P}')
    fixed = fixed.replace('\\WW', '\\mathcal{W}')

    return fixed

In [ ]:
for i, latex_math_node in enumerate(math_nodes[:100]):
    latex = latex_math_node.latex_verbatim()
    latex_fixed = fix_latex(latex)
    math_display_object = Math(latex_fixed)

    # print(i)
    # print(math_display_object._repr_latex_())

    display(math_display_object)

### Classify

In [5]:
from decouple import config

from math_rag.infrastructure.inference.llms import LLM


OPENAI_BASE_URL = config('OPENAI_BASE_URL')
OPENAI_API_KEY = config('OPENAI_API_KEY')

In [6]:
llm = LLM(model='gpt-4o', base_url=OPENAI_BASE_URL, api_key=OPENAI_API_KEY)

In [5]:
# math_expr = math_nodes[13].latex_verbatim()  # 13
math_expr = '$\\mu(x)=\\frac{1}{2^n}$'

In [ ]:
prompt = f"""
You are a mathematical expression classifier.
Given a mathematical expression, classify it in one of 4 given classes:
- constant
- variable
- formula
- other

Return a class only!

Mathematical expression:
{math_expr}

Class:
"""

In [7]:
from openai import NOT_GIVEN
from openai.types.chat import ChatCompletion


def get_prompt(math_expr: str) -> str:
    return f"""
You are a mathematical expression classifier.
Given a mathematical expression, classify it in a single class regarding STRUCTURE of the expression.
Class must be a single word.

Return a class only!

Mathematical expression:
{math_expr}

Class:
"""


use_json = False


async def get_completion(prompt: str) -> ChatCompletion:
    return await llm.client.chat.completions.create(
        model='gpt-4o',
        messages=[{'role': 'user', 'content': prompt}],
        response_format={'type': 'json_object'} if use_json else NOT_GIVEN,
        logprobs=True,
        temperature=0.0,
        top_logprobs=5,
    )

In [8]:
math_expressions_by_category = (
    await math_expression_repository.get_math_expressions_by_category(32)
)

In [ ]:
from math_rag.core.models import MathExpressionPrediction


completions: list[ChatCompletion] = []
predictions: list[MathExpressionPrediction] = []

from time import sleep


for category, math_expressions in math_expressions_by_category.items():
    for math_expression in math_expressions:
        if math_expression.id in processed_ids:
            continue

        prompt = get_prompt(math_expression.latex)
        completion = await get_completion(prompt)

        prediction = MathExpressionPrediction(
            math_expression_id=math_expression.id,
            value=completion.choices[0].message.content,
        )
        completions.append(completion)
        predictions.append(prediction)

        # await math_expression_prediction_repository.insert_math_expression_predictions([prediction])
        sleep(1)

In [ ]:
await math_expression_prediction_repository.insert_math_expression_predictions(
    predictions
)

In [15]:
import json


with open('../tmp/completions.json', 'w') as json_file:
    json.dump([x.to_dict() for x in completions], json_file, indent=4)

In [ ]:
from uuid import UUID


class UUIDEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, UUID):
            return str(obj)

        return super().default(obj)


with open('../tmp/predictions.json', 'w') as json_file:
    json.dump(
        [x.model_dump() for x in predictions], json_file, indent=4, cls=UUIDEncoder
    )

In [ ]:
completion = get_completion()

completion.choices[0].message.content

'Function'

In [57]:
import numpy as np


for x in completion.choices[0].logprobs.content:
    for y in x.top_logprobs:
        print(f'"{y.token}": {np.exp(y.logprob)}')

    print('------')
    print(x.token)
    print(x.logprob)
    print(np.exp(x.logprob))

"formula": 0.9999996871837189
" formula": 9.931194312156244e-08
"Formula": 7.734421907141565e-08
"_formula": 9.237449661970594e-09
"form": 2.061153622438558e-09
------
formula
-3.1281633e-07
0.9999996871837189


In [ ]:
# TODO
# - description for each class
# - how to determine classes?
# - do names need to take a single token?

In [ ]:
from enum import Enum


class MathExpressionCategory(str, Enum):
    EQUALITY = 'equality'
    INEQUALITY = 'inequality'
    CONSTANT = 'constant'
    VARIABLE = 'variable'
    OTHER = 'other'

In [5]:
predictions = (
    await math_expression_prediction_repository.get_math_expression_predictions(1024)
)

In [6]:
expressions = [
    await math_expression_repository.get_math_expression_by_id(
        prediction.math_expression_id
    )
    for prediction in predictions
]

In [12]:
for expression, prediction in zip(expressions, predictions):
    # latex_fixed = fix_latex(latex)
    math_display_object = Math(expression.latex)

    print(prediction.value)
    display(math_display_object)
    print('-----')

Tuple


<IPython.core.display.Math object>

-----
set


<IPython.core.display.Math object>

-----
Variable


...

-----
function


<IPython.core.display.Math object>

-----
Variable


<IPython.core.display.Math object>

-----
